In [ ]:
import os
import zipfile
import glob
import ast
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import pyiast
plt.style.use("seaborn-v0_8")


In [ ]:
# Locate and extract Project 2.zip

# Potential zip paths (adjust depending on where your notebook lives)
possible_zips = [
    "Project 2.zip",
    os.path.join("project", "Project 2.zip"),
    "/mnt/data/Project 2.zip",
    os.path.join("/mnt/data", "Project 2.zip")
]

zip_path = None
for p in possible_zips:
    if os.path.exists(p):
        zip_path = p
        break

if zip_path is None:
    raise FileNotFoundError("Could not find 'Project 2.zip'. Put it in the notebook folder or update the path.")

# Extract target (choose a clear folder)
extract_path = os.path.join("project_data_from_zip")
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(extract_path)
    print("Extracted zip to:", extract_path)
else:
    print("Extraction folder already exists:", extract_path)

# Show top-level files (for user awareness)
for root, dirs, files in os.walk(extract_path):
    # print only first-level summary
    break

print("Top-level folders under extract:", os.listdir(extract_path)[:50])


# Q1 — Pore analysis parsing


In [ ]:

def parse_kv_csv(path):
    """
    Parse a CSV of Key,Value rows into a dict. Tries to interpret numeric / list values.
    """
    d = {}
    with open(path, "r", encoding="utf-8") as fh:
        for line in fh:
            parts = line.rstrip("\n").split(",", 1)
            if len(parts) == 2:
                key = parts[0].strip()
                val = parts[1].strip()
                # try to convert lists / numbers
                try:
                    if val.startswith("[") or val.startswith("("):
                        d[key] = ast.literal_eval(val)
                    else:
                        # numeric detection (handles scientific notation)
                        try:
                            d[key] = float(val)
                        except Exception:
                            d[key] = val
                except Exception:
                    d[key] = val
    return d

pore_folder = None
for candidate in glob.glob(os.path.join(extract_path, "**"), recursive=False):
    # try to find a folder named "Pore analysis" anywhere under the extraction root
    candidate = os.path.join(extract_path, "Project 2", "Pore analysis")
    break

if os.path.isdir(candidate):
    pore_folder = candidate
else:
    # fallback: find any folder with "Pore" in name
    matches = [p for p in glob.glob(os.path.join(extract_path, "**", "*Pore*"), recursive=True) if os.path.isdir(p)]
    pore_folder = matches[0] if matches else None

if pore_folder is None:
    raise FileNotFoundError("Could not find 'Pore analysis' folder inside the extracted zip.")

pore_files = sorted(glob.glob(os.path.join(pore_folder, "*.csv")))
pore_rows = []
for pf in pore_files:
    d = parse_kv_csv(pf)
    # collect keys safely:
    name = Path(pf).stem.replace(" pore analysis", "").strip()
    density = d.get("Density") or d.get("Density_g_cm^3") or None
    asa = d.get("ASA_A^2") or d.get("ASA_A^2_unit") or d.get("ASA_m^2/cm^3") or d.get("Accessible_surface_area")
    poav = d.get("POAV_A^3") or d.get("POAV_cm^3/g") or d.get("POAV_cm^3/g")
    porosity = d.get("Porosity") or d.get("POAV_Volume_fraction") or d.get("Porosity_fraction")
    pore_rows.append({
        "Structure": name,
        "Density (g/cm^3)": float(density) if density not in (None,"") else np.nan,
        "ASA (Å² or m²/cm³)": asa,
        "POAV (Å³ or cm³/g)": poav,
        "Porosity (fraction)": porosity
    })

df_pore = pd.DataFrame(pore_rows).set_index("Structure")
df_pore


# Q1 Results — Pore properties

The table above shows parsed pore-analysis results (density, ASA, POAV, porosity).



# Q2 — Henry coefficients for CO₂ and N₂

We parse the CO₂ and N₂ outputs (these CSVs often include a `henry_coefficient_average` key).
If a Henry coefficient is not present, the script falls back to a low-pressure linear fit
on the supplied pure-component isotherm near the origin (first few points).
All Henry coefficients are converted to **mmol/g/bar** for easy comparison:

- Original unit: mol/kg/Pa  → multiply by 1e5 to obtain mol/kg/bar.
- mol/kg/bar is numerically equal to mmol/g/bar (1 mol/kg = 1 mmol/g).


In [ ]:
def load_gas_csvs(root_extract, gas_dirname):
    """
    Return dict: {structure_name: parsed_dict}
    """
    base = os.path.join(root_extract, "Project 2", gas_dirname)
    if not os.path.isdir(base):
        # try direct subfolders
        matches = [p for p in glob.glob(os.path.join(root_extract, "**", gas_dirname), recursive=True) if os.path.isdir(p)]
        base = matches[0] if matches else base
    files = sorted(glob.glob(os.path.join(base, "*.csv")))
    parsed = {}
    for f in files:
        name = Path(f).stem.replace(" ", "").replace("_", " ").strip()
        parsed[name] = parse_kv_csv(f)
    return parsed

co2_parsed = load_gas_csvs(extract_path, "CO2")
n2_parsed = load_gas_csvs(extract_path, "N2")

def extract_isotherm(parsed_dict):
    """
    From the parsed CSV dict, extract pressure (bar) and loading (mmol/g or mol/kg).
    Returns a dict with 'pressure' and 'loading' arrays and the loading unit.
    """
    iso = parsed_dict.get("isotherm")
    if iso:
        # isotherm stored as stringified dict -> convert
        iso_d = ast.literal_eval(iso) if isinstance(iso, str) else iso
        pressure = np.array(iso_d.get("pressure", []), dtype=float)
        # attempts for loading key
        loading = np.array(iso_d.get("loading_absolute_average", iso_d.get("loading_average", iso_d.get("loading", []))), dtype=float)
        loading_unit = iso_d.get("loading_absolute_unit", iso_d.get("loading_unit", ""))
        return {"pressure": pressure, "loading": loading, "unit": loading_unit}
    else:
        return None

def get_henry_from_parsed(parsed):
    """
    Return (henry_mmol_g_bar, source_flag)
    source_flag tells whether it's from 'henry_field' or 'lowP_fit' or None
    """
    # try direct field
    kh = parsed.get("henry_coefficient_average")
    unit = parsed.get("henry_coefficient_unit", "")
    if kh is not None:
        # unit (expected mol/kg/Pa) -> convert to mmol/g/bar by multiplying by 1e5
        try:
            kh_val = float(kh) * 1e5
            return kh_val, "henry_field"
        except Exception:
            pass
    # fallback: if isotherm present, compute low-pressure slope (first few points)
    iso = extract_isotherm(parsed)
    if iso and len(iso["pressure"]) >= 3:
        # ensure pressure in bar (they are already in bar in these files)
        p = iso["pressure"]
        q = iso["loading"]
        # choose first N points with p <= 0.5 bar for low-pressure linear fit
        idx = np.where(p <= 0.6)[0]
        if len(idx) >= 2:
            idx = idx[:min(len(idx), 6)]
            # linear fit q = k * p -> slope is k (units: same as q per bar)
            slope, intercept = np.polyfit(p[idx], q[idx], 1)
            # If q units are mol/kg -> slope is mol/kg/bar -> same numeric as mmol/g/bar
            return float(slope), "lowP_fit"
    return np.nan, "no_data"

# Build Henry table
rows = []
all_structures = sorted(set(list(co2_parsed.keys()) + list(n2_parsed.keys())))
for s in all_structures:
    co2_dat = co2_parsed.get(s, {})
    n2_dat = n2_parsed.get(s, {})
    kh_co2, src_co2 = get_henry_from_parsed(co2_dat) if co2_dat else (np.nan, None)
    kh_n2, src_n2 = get_henry_from_parsed(n2_dat) if n2_dat else (np.nan, None)
    rows.append({"Structure": s, "K_H_CO2 (mmol/g/bar)": kh_co2, "K_H_N2 (mmol/g/bar)": kh_n2, 
                 "CO2_KH_source": src_co2, "N2_KH_source": src_n2,
                 "KH_ratio_CO2/N2": (kh_co2 / kh_n2) if (kh_n2 and not math.isnan(kh_n2)) else np.nan})
df_henry = pd.DataFrame(rows).set_index("Structure").sort_values("K_H_CO2 (mmol/g/bar)", ascending=False)
df_henry


# Q2 Results — Henry coefficients and ranking

The table above lists the Henry coefficients (converted to **mmol/g/bar**) for CO₂ and N₂ and the source used:
- `henry_field` indicates the file contained a direct Henry coefficient (e.g., from a Widom insertion).
- `lowP_fit` indicates the coefficient was estimated from the low-pressure slope of the isotherm.

We sort by **K_H(CO₂)** to rank the materials by CO₂ affinity.


# Q3 — Pure-component isotherms (CO₂ and N₂)

We extract the isotherm arrays from each structure's CSV and plot:
- CO₂ isotherms (adsorbed amount vs pressure)
- N₂ isotherms (adsorbed amount vs pressure)

The isotherm data typically contains `pressure` and a loading array (e.g., `loading_absolute_average`).


In [ ]:
def build_isotherm_df(parsed_dict):
    iso = extract_isotherm(parsed_dict) if parsed_dict else None
    if iso is None:
        return None
    p = np.array(iso["pressure"], dtype=float)
    q = np.array(iso["loading"], dtype=float)
    unit = iso.get("unit", "")
    return pd.DataFrame({"pressure_bar": p, "loading": q, "unit": unit})

# Build combined DF for CO2 and N2
co2_iso_dfs = {}
n2_iso_dfs = {}
for s, parsed in co2_parsed.items():
    df_iso = build_isotherm_df(parsed)
    if df_iso is not None:
        co2_iso_dfs[s] = df_iso.sort_values("pressure_bar")

for s, parsed in n2_parsed.items():
    df_iso = build_isotherm_df(parsed)
    if df_iso is not None:
        n2_iso_dfs[s] = df_iso.sort_values("pressure_bar")

# Plot CO2 isotherms
plt.figure(figsize=(8,6))
for s, df in co2_iso_dfs.items():
    plt.plot(df["pressure_bar"], df["loading"], label=s)
plt.xlabel("Pressure (bar)")
plt.ylabel("Loading (unit reported in files)")
plt.title("Pure CO2 isotherms at 25 °C")
plt.legend(loc='best', fontsize='small', ncol=2)
plt.xlim(0, None)
plt.grid(True)
plt.show()

# Plot N2 isotherms
plt.figure(figsize=(8,6))
for s, df in n2_iso_dfs.items():
    plt.plot(df["pressure_bar"], df["loading"], label=s)
plt.xlabel("Pressure (bar)")
plt.ylabel("Loading (unit reported in files)")
plt.title("Pure N2 isotherms at 25 °C")
plt.legend(loc='best', fontsize='small', ncol=2)
plt.xlim(0, None)
plt.grid(True)
plt.show()


# Q3 Notes

- Unit labels appear in each isotherm CSV; the plotting code uses the raw loading numbers and reports the unit in the DataFrame.
- If you want the vertical axis in mmol/g explicitly, we can standardize units (e.g. convert mol/kg → mmol/g).


# Q4 — CO₂ Working Capacity (WC)

Working capacity is computed as:
\[
WC = q_{adsorption} - q_{desorption}
\]
where:
- q_adsorption is the CO₂ loading at 1.0 bar (adsorption step),
- q_desorption is the CO₂ loading at 0.2 bar (desorption step).

We interpolate the isotherms to get q at these pressures.


In [ ]:
wc_rows = []
for s, df in co2_iso_dfs.items():
    p = df["pressure_bar"].values
    q = df["loading"].values
    # ensure monotonic p for interpolation
    sort_idx = np.argsort(p)
    p_sorted = p[sort_idx]
    q_sorted = q[sort_idx]
    # interpolation (extrapolate with nearest if needed)
    def interp_q_at(pressure):
        if pressure <= p_sorted.min():
            return q_sorted[0]
        if pressure >= p_sorted.max():
            return q_sorted[-1]
        return float(np.interp(pressure, p_sorted, q_sorted))
    q_ads = interp_q_at(1.0)
    q_des = interp_q_at(0.2)
    wc = q_ads - q_des
    wc_rows.append({"Structure": s, "q_ads (1.0 bar)": q_ads, "q_des (0.2 bar)": q_des, "WC (q_ads - q_des)": wc})

df_wc = pd.DataFrame(wc_rows).set_index("Structure").sort_values("WC (q_ads - q_des)", ascending=False)
df_wc


# Q4 Results — Working capacities

The table above shows:
- q_ads at 1.0 bar,
- q_des at 0.2 bar,
- the computed WC for each structure.

You can use `df_wc.join(df_henry).join(df_pore)` to create a master table that contains
pore metrics, Henry coefficients, and WC for side-by-side comparison.
